## Проект по SQL

**Описание проекта**


Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.


Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.


Таблица `books` - Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.


Таблица `authors` - Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

Таблица `publishers` - Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

Таблица `ratings` - Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица `reviews` - Содержит данные о пользовательских обзорах на книги:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзора.


In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры

db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas

In [3]:
def disp(value):
    query = ('''SELECT * FROM {} LIMIT 5'''.format(value))
    con=engine.connect()
    display(pd.io.sql.read_sql(sql=text(query), con = con))
    return pd.io.sql.read_sql(sql=text(query), con = con)

books = disp('books')
authors = disp('authors')
publishers = disp('publishers')
ratings = disp('ratings')
reviews = disp('reviews')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...



### Задание 1. Посчитайте, сколько книг вышло после 1 января 2000 года;

In [4]:
query = '''

SELECT COUNT( book_id )

FROM books
WHERE publication_date > '2000-01-01'

'''

post_view_sum = pd.read_sql_query(query, con=engine) 
post_view_sum

,count
0,819


### Задание 2. Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [5]:
"""
query = '''
    SELECT b.book_id,
           COUNT(r.review_id) AS review_count,
           AVG(rat.rating) AS average_rating

    FROM books AS b JOIN
         reviews AS r ON b.book_id = r.book_id JOIN
         ratings AS rat ON b.book_id = rat.book_id
    GROUP BY 1
    ORDER BY 2 DESC

'''

post_view_sum = pd.read_sql_query(query, con=engine) 
post_view_sum
"""

#Предыдущий код закомментировала

query = '''
WITH tab1 AS(

    SELECT book_id,
           AVG(rating) AS average_rating
    FROM ratings
    GROUP BY 1
    ),
tab2 AS
(
    SELECT book_id,
           COUNT(review_id) AS review_count
    FROM reviews
    GROUP BY 1
    ORDER BY 2 DESC
)

SELECT b.title,
       tab1.average_rating,
       tab2.review_count
FROM tab1 LEFT JOIN tab2 ON tab1.book_id = tab2.book_id JOIN
books AS b ON b.book_id = tab1.book_id
ORDER BY 3 DESC NULLS LAST 
'''

post_view_sum = pd.read_sql_query(query, con=engine) 
post_view_sum

,title,average_rating,review_count
0,Twilight (Twilight #1),3.662500,7.0
1,Water for Elephants,3.977273,6.0
2,The Glass Castle,4.206897,6.0
3,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,6.0
4,The Curious Incident of the Dog in the Night-Time,4.081081,6.0
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,3.666667,NaN
996,The Natural Way to Draw,3.000000,NaN
997,The Cat in the Hat and Other Dr. Seuss Favorites,5.000000,NaN
998,Essential Tales and Poems,4.000000,NaN


### 3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [6]:
query = '''
WITH tab1 AS(
                WITH tab AS(  
                            SELECT *
                            FROM books 
                            WHERE num_pages > 50
                            )
                SELECT publisher_id,
                       COUNT(book_id) as book_count
                FROM tab
                GROUP BY publisher_id
                ORDER BY 2 DESC
                LIMIT 1
    )       
SELECT publisher,
       book_count
FROM tab1 INNER JOIN publishers ON tab1.publisher_id = publishers.publisher_id

'''

post_view_sum = pd.read_sql_query(query, con=engine) 
post_view_sum

,publisher,book_count
0,Penguin Books,42


### 4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [7]:
query = '''
WITH tab1 AS(
WITH tab AS(
SELECT book_id,
       AVG(rating) AS average_rating
FROM ratings
GROUP BY 1
HAVING COUNT(rating) >= 50
)
SELECT b.author_id,
       AVG(tab.average_rating) AS author_average_rating
FROM tab JOIN books AS b ON b.book_id = tab.book_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1)
SELECT tab1.author_id,
       authors.author,
       tab1.author_average_rating
FROM tab1 INNER JOIN authors ON tab1.author_id = authors.author_id
        
'''

post_view_sum = pd.read_sql_query(query, con=engine) 
post_view_sum

,author_id,author,author_average_rating
0,236,J.K. Rowling/Mary GrandPré,4.283844


### 5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.


In [8]:
query = '''
WITH tab1 AS
        (WITH tab AS ( 
                SELECT username 
                FROM ratings
                GROUP BY 1
                HAVING COUNT(rating) > 48
                )
          SELECT tab.username,
          COUNT(review_id) AS count_review
FROM tab INNER JOIN reviews ON tab.username = reviews.username
GROUP BY 1
) 
SELECT AVG(count_review) AS users_average_reviews_count
FROM tab1

'''
post_view_sum = pd.read_sql_query(query, con=engine) 
post_view_sum

,users_average_reviews_count
0,24.0
